# Agumentación

## Introducción

Al existir un desbalance muy grande entre las clases se evaluará el uso técnicas para solucionarlo.

Se descarta duplicar ejemplos de la clase minoritaria para evitar posibles sobreajustes y en un principio también se descarta disminuir a las clases mayoritarias para no perder variabilidad.

Se explorará el uso de modelos de lenguaje para la generación de texto, tanto en versiones que se puedan ejecutar localmente como versiones pagas accesibles por API.

También se usarán modelos para el parafraseo de datos existentes.

Adicionalmente se hará la traducción a otro idioma y luego de vuelta al español (backtranslation) y se usarán sinónimos para obtener cambios del texto original preservando el significado.

Podría aplicarse SMOTE (Synthetic Minority Over-sampling Technique) una vez obtenidos los embeddings pero al no poder evaluar el resultado en forma de texto se plantea su uso en trabajos futuros.

## Lectura de datos y análisis de distribución de clases

In [1]:
import pandas as pd
df = pd.read_csv("mensajes_preprocesados.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7050 entries, 0 to 7049
Data columns (total 31 columns):
 #   Column                                                                            Non-Null Count  Dtype 
---  ------                                                                            --------------  ----- 
 0   Unnamed: 0                                                                        7050 non-null   int64 
 1   id_participante                                                                   7050 non-null   int64 
 2   id_mensaje                                                                        7050 non-null   int64 
 3   enviado                                                                           7050 non-null   object
 4   tipo                                                                              7048 non-null   object
 5   subtipo                                                                           358 non-null    object
 6   recibido

In [2]:
# distribución de clases
df["clase"].value_counts()

clase
respuesta a consejo o pregunta    2084
agradecimiento                    2046
optin                             1833
pregunta                           226
saludo                             214
turno                              132
baja                               113
autorespuesta                      111
glicemia                            86
equivocado                          72
medicación                          68
queja                               23
Name: count, dtype: int64

In [3]:
# es de interés augmentar las clases "glicemia", "equivocado", "queja" y la combinación de "turno" y "medicacion"
# se explora con "glicemia"

In [4]:
textos_augmentados = {}
textos_augmentados["glicemia"] = []

## Generación de texto

### Modelo local

In [5]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [6]:
import random
from transformers import pipeline, set_seed

# Inicializar el generador de texto usando el modelo en español entrenado con datos de la Biblioteca Nacional de España
# (la descarga demora mucho, son 3.13 GB con un ancho de banda pobre)
generator = pipeline('text-generation', model='PlanTL-GOB-ES/gpt2-large-bne', device = device)
set_seed(42)

In [7]:
# ejemplo de uso con diferentes temperaturas donde se comienza la frase y el modelo la continúa
prompt = "Al resultado del análisis de hemoglobina glicosilada no lo tengo porque"
print(prompt)
for temperatura in range(2,8):
    print("------------------")
    print(f"Temperatura: {temperatura/10}")
    for result in generator(prompt, num_return_sequences=5, temperature=temperatura/10, top_p=0.9):
        print(result['generated_text'][len(prompt):].strip()) # Extraer solo la parte generada

Al resultado del análisis de hemoglobina glicosilada no lo tengo porque
------------------
Temperatura: 0.2


C:\Users\aciga\anaconda3\envs\nlp\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:650: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


no me lo han hecho, pero si que me han dicho que es normal.
no lo he hecho, pero si que he visto que la hemoglobina glicosilada es un marcador de la función renal.
no me ha dado tiempo a hacer el análisis de sangre, pero si que he visto que hay una serie de factores que influyen en la hemoglobina glicos
no me ha dado tiempo a leerlo, pero me ha gustado mucho.
no me ha dado tiempo a hacer la prueba de la hematopoyesis, pero si que he visto que hay una pequeña cantidad de hemoglobina en
------------------
Temperatura: 0.3
no me lo han hecho, pero si que me han dicho que es normal, que no es nada grave.
no lo he encontrado en ningún sitio, pero si que me han dicho que es una proteína que se encuentra en la sangre y que se produce por la acción de la
no tengo.
no lo he hecho nunca, pero el resultado de la hemoglobina glicosilada es el mismo que el de la hemoglobina normal, así
no me da la vida, pero si que me da la vida, por eso me gusta tanto la cerveza.
------------------
Temperatura: 0.

In [8]:
# en general estas respuestas no parecen muy creibles, con temperatura alta se va de tema, se utiliará temperatura 0.4 con más frases iniciales

In [9]:
frases = [
    "Al resultado del análisis de hemoglobina glicosilada no lo tengo porque",
    "No conozco el valor de azúcar en sangre",
    "Me falta el control de azúcar desde",
    "Ignoro cuál es el valor del análisis de azúcar en sangre porque",
    "Sobre mis valores de azúcar en sangre puedo decir que"
]

In [10]:
for frase in frases:
    print("=====================")
    print(frase + ":")
    print("-------------")
    generados = generator(frase, num_return_sequences=10, temperature=0.4)
    for i in range(len(generados)):
      generado = generados[i]['generated_text'][len(frase):].strip()[:80]  # Extraer solo la parte generada y muestra hasta 80 caracteres
      print(generado)
      textos_augmentados["glicemia"].append({'origen':'gpt2-large-bne', 'texto':generado})


Al resultado del análisis de hemoglobina glicosilada no lo tengo porque:
-------------
no me he puesto a ello, pero sí que me ha llamado la atención el dato de que la 
no me encuentro muy bien, pero si que he notado que tengo más volumen de lo norm
no me ha llegado, pero si que he visto que la hemoglobina glicosilada no es la m
no lo he leído, pero si lo tuviera lo pondría en la lista de "libros pendientes"
no he tenido la oportunidad de hacerlo, pero si se puede saber que la sangre de 
no es un tema que me interese, pero si que me ha gustado mucho el resultado.
no me interesa, pero el de la hemoglobina glicosilada sí que lo tengo.
no tengo el glucómetro.
no lo he conseguido.
no me ha dado tiempo a hacer el análisis y no tengo el resultado.
No conozco el valor de azúcar en sangre:
-------------
, pero si en la orina, por lo que me han comentado, debe ser elevado.
, pero si es un tema de salud, es una buena noticia.
, pero sí sé que hay alimentos que contienen azúcar en su composición y

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


no lo he leído en ningún lado.
, aunque es muy difícil de cuantificar, las cifras son muy similares a las de lo
me resulta difícil de entender.
no lo he hecho.
no lo he hecho nunca.
en la práctica no se suele realizar, pero lo que sí sé es que hay una serie de p
no me lo han enseñado en la universidad, pero si se puede medir con una muestra 
no lo he encontrado.
no es el único que se ha hecho en el mundo y porque lo que se ha hecho en el Rei
no he podido encontrar ninguna información al respecto en la literatura médica.
Sobre mis valores de azúcar en sangre puedo decir que:
-------------
son los mismos que los de cualquier otro alimento.
no me gustan, no me gusta, no me gusta, no me gusta, no me gusta, no me gusta, n
no me gusta el azúcar, y que el azúcar no es malo, pero sí que me gusta el azúca
soy una persona muy positiva, que me gusta ayudar a los demás, que soy muy socia
soy una persona muy tranquila y muy positiva.
no soy muy dado a los edulcorantes, pero si que me gusta mucho el

In [11]:
# Los resultados no resultan similares a los mensajes recibidos reales por lo que no se se usarán

### Mistral AI

In [12]:
!pip install mistralai

In [13]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

api_key = "eOLBUzZrqcYBLQeLc8AhcBxnQ05bjhW6"
model = "mistral-large-latest"

client = MistralClient(api_key=api_key)


In [14]:
prompt = "Generar en español 20 respuestas negativas que daría un paciente a un doctor que le pregunta si conoce sus valores de azúcar en sangre."

In [15]:
chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="user", temperature = 0.4, content=prompt)]
)

print(chat_response.choices[0].message.content)

1. "Lo siento, doctor, no tengo ni idea de cuáles son mis valores de azúcar en sangre."
2. "No, no conozco mis niveles de azúcar en sangre, nunca me los han medido."
3. "Lamentablemente, no sé cuáles son mis valores de azúcar en sangre."
4. "No he prestado atención a eso, doctor, así que no sé cuáles son mis niveles."
5. "No he hecho los análisis necesarios para conocer mis valores de azúcar en sangre."
6. "No recuerdo haberlos revisado, así que no puedo decirle cuáles son mis niveles."
7. "Desafortunadamente, no he estado al tanto de mis valores de azúcar en sangre."
8. "No, doctor, no he tenido la oportunidad de chequear mis niveles de azúcar."
9. "No he recibido información sobre mis valores de azúcar en sangre, así que no puedo responder."
10. "No he estado monitoreando mis niveles de azúcar, así que no tengo esa información."
11. "No he tenido un control regular de mis niveles de azúcar, así que no los conozco."
12. "No he prestado suficiente atención a mis niveles de azúcar, por 

In [16]:
prompt = prompt + "Utilizar frases cortas, informales en pasado simple. No denotar culpa. Utilizar emoticones en la mitad de las respuestas."

In [17]:
chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="user", temperature = 0.4, content=prompt)]
)

print(chat_response.choices[0].message.content)

1. No, doctor, nunca los revisé.
2. Lo siento, no los tengo presentes.
3. No me acuerdo, ¿los chequé alguna vez?
4. No los conozco, doctor. 😐
5. Nunca me los dijeron, ¿debería saberlos?
6. No los sé, ¿son importantes?
7. No, nunca me los controlé. 🤔
8. No los recuerdo, doctor. 🤷‍♀️
9. No, nunca me los dijeron. 😶
10. No los tengo a mano, doctor.
11. No, no los conozco. 😕
12. No los revisé, ¿es malo?
13. No los sé, doctor. 🙁
14. No, nunca me los chequé. 😟
15. No los conozco, ¿debería?
16. No, doctor, no los sé. 😞
17. No los tengo, ¿los necesita?
18. No los revisé, doctor. 😓
19. No los conozco, lo siento. 😖
20. No, nunca los supe. 😔


In [18]:
prompt = prompt + "Agregar como posibles causas no haber hecho el análisis, la falta de tiempo y la poca disponibilidad de turnos o insumos como ser cintas o medidor."

In [19]:
chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="user", temperature = 0.4, content=prompt)]
)

print(chat_response.choices[0].message.content)

1. "Lo siento, no lo sé, no tuve tiempo de hacerme el análisis." 🕒
2. "No pude chequearlo, se me acabaron las cintas." 📉
3. "No he podido medir mi azúcar, no hay turnos disponibles." 📅
4. "No lo conozco, el medidor se dañó." 🛠️
5. "No he podido verificarlo, estuve muy ocupado." 🏃‍♂️
6. "No sé mis valores, no encontré cintas reactivas." 🔍
7. "No lo he medido, no he podido conseguir un medidor." 💊
8. "No tengo idea, no he tenido la oportunidad." 😕
9. "No lo sé, no he podido ir al laboratorio." 🏥
10. "No he podido chequearlo, no tuve el tiempo necesario." ⏱️
11. "No sé mis valores, se me olvidó hacerlo." 😐
12. "No lo he revisado, no había insumos disponibles." 📝
13. "No he podido medirlo, no he podido conseguir turno." 📆
14. "No sé mis valores, el medidor no funciona bien." 🛠️
15. "No lo he chequeado, no he podido comprar cintas." 💰
16. "No lo sé, no he podido hacerme el análisis." 💉
17. "No he podido medirlo, no he tenido la posibilidad." 😞
18. "No lo conozco, no he podido encontrar un l

In [20]:
prompt = prompt + "Utlizar expresiones y abreviaturas informales. Agregar alguna falta de ortografía o error de tipeado común en celulares"

In [21]:
chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="user", temperature = 0.4, content=prompt)]
)

print(chat_response.choices[0].message.content)

1. "Ni idea, doc. No tuve tiempo de hacerme el análisis." 😤
2. "No pude, se agotaron las cintas en la farmacia." 🙁
3. "Lo siento, no alcancé a chequear mis valores." 😕
4. "Nada, no hubo turnos para el laboratorio." 😞
5. "Cero tiempo, doctor. No pude ir a hacerme el examen." 😖
6. "No me acuerdo, creo que no lo hice." 🤔
7. "No pude, no me dieron turno en el lab." 😒
8. "No sé, no encontré el medidor." 😟
9. "No tuve chance, se me olvidó." 😐
10. "No, no pude conseguir las cintas reactivas." 😣
11. "No chekee mis valores, no hubo tiempo." 😵
12. "No, no había turnos. Lo siento." 😔
13. "No me acuerdo si lo hice, doctor." 😕
14. "No pude, no había insumos en la farmacia." 😞
15. "No, no me alcanzó el tiempo." 😫
16. "No sé mis valores, no encontré el medidor." 🤯
17. "No, no hubo turnos en el lab. Lo siento." 😟
18. "No pude hacermelo, no había cintas." 😖
19. "No, no me dio el tiempo. Perdón." 🙁
20. "No sé, no conseguí el medidor. Lo siento." 😕


In [22]:
prompt = 'Generar en español 50 respuestas negativas que daría un paciente diabético al que se le pregunta si conoce sus valores de azúcar en sangre. Utilizar frases cortas, en tono informal, en pasado simple. No denotar culpa. Posibles excusas: desconocerlo, no recordarlo, olvido, falta de tiempo, poca disponibilidad de turnos o insumos como ser cintas o medidor. Usar expresiones y abreviaturas informales rioplatenses. Agregar faltas de ortografía o errores de tipeado común en celulares' 

In [23]:
system_prompt = "Responder como si fuera mensaje de Whatsapp en español rioplatense usando tono informal, en presente o pasado simple. Incluir emoticones, abreviaturas, faltas de ortografía y errores de tipeado. Cada respuesta en una línea sin enumerar."
user_prompt = "Generar en español 50 respuestas negativas que daría un paciente diabético al que se le pregunta si conoce sus valores de azúcar en sangre.  Posibles excusas: desconocerlo, no recordarlo, olvido, falta de tiempo, poca disponibilidad de turnos, no tener cintas o problemas con el medidor. No denotar culpa."

chat_response = client.chat(
    model=model,
    messages=[
        ChatMessage(role="system", content=system_prompt),
        ChatMessage(role="user", temperature = 0.4, content=user_prompt)
             ]
)

print(chat_response.choices[0].message.content)

1. No che, no me acuerdo ahora los valores 🤦‍♂️
2. Uff, no tengo idea, el medidor anda mal últimamente 😕
3. Ay, no sé, se me olvidó medirme hoy 🙈
4. No tuve tiempo de chequear hoy, lo juro ⏰
5. Nah, no me acuerdo, tengo la cabeza en otra cosa 🧠
6. No pude, se me terminaron las tiras 😖
7. No me acuerdo ahora, pero mañana te digo, prometo 🤞
8. No sé, no me sentí bien y no quise medirme 🤢
9. No me acuerdo, hace rato que no me controlo 😬
10. No, no me dio el tiempo, anduve todo el día corriendo 🏃‍♂️
11. No sé, no me acuerdo si lo hice o no 🤔
12. No pude, no tenía el medidor a mano 🙁
13. No sé, ayer comí algo raro y no quise medirme 🍔
14. No, no me da el cuero, estoy cansado de pincharme 💉
15. No sé, tengo la cabeza en otra, perdoná 😐
16. No, no me acuerdo, ando medio despistado últimamente 😵
17. No pude, el medidor no anda bien y no me da los valores 😠
18. No sé, no tengo ganas de estar todo el día midiéndome 😒
19. No, no me acuerdo, ayer fue un día muy loco 🤪
20. No sé, no me sentí bien y

### ChatGPT

In [26]:
from openai import OpenAI
import os

os.environ["OPENAI_API_KEY"] = "sk-proj-8vhbvoWr2DeqeOh8aeSwT3BlbkFJljgG4JFIlyvMaMWqulUI"

def query_chat_gpt(user_prompt="", system_prompt="", model="gpt-4o-mini-2024-07-18", temperature=0.5):
    result = None
    mensajes = []    
    if user_prompt!="":
        mensajes.append({"role": "user", "content": user_prompt})
    if system_prompt!="":
        mensajes.append({"role": "system", "content": system_prompt})
    if len(mensajes)>0:
        client = OpenAI()
        result = client.chat.completions.create(
          model=model,
          temperature=temperature,
          messages=mensajes
        )
    return result

In [27]:
system_prompt = "Responder como si fuera mensaje de Whatsapp en español rioplatense usando tono informal, en presente o pasado simple. Incluir emoticones, abreviaturas, faltas de ortografía y errores de tipeado. Cada respuesta en una línea sin enumerar."
user_prompt = "Generar en español 50 respuestas negativas que daría un paciente diabético al que se le pregunta si conoce sus valores de azúcar en sangre.  Posibles excusas: desconocerlo, no recordarlo, olvido, falta de tiempo, poca disponibilidad de turnos, no tener cintas o problemas con el medidor. No denotar culpa."

respuesta = query_chat_gpt(user_prompt, system_prompt, temperature=0.6)

print(respuesta.choices[0].message)

ChatCompletionMessage(content='no, la verdad no sé mis valores 😅  \nno me acuerdo, hace tiempo que no me controlo 🤷\u200d♂️  \nuy, se me pasó che, no lo miré últimamente 😬  \nno tengo tiempo, siempre estoy a mil 💨  \nno conseguí turno en el médico, una garrón 😩  \nno tengo cintas para el medidor, así que no puedo 😕  \nme olvidé de chequearlo, la vida me lleva 😅  \nno tengo el medidor a mano ahora 🤷\u200d♀️  \nhace mucho que no lo miro, no me acuerdo 😅  \nno sé, me faltó ir a la farmacia a comprar cintas 😩  \nno tengo idea, me olvidé de preguntar 😬  \nno pude ir al doc, así que no tengo datos 😔  \nno sé, estoy en otra cosa y no me acuerdo 😅  \nno tengo tiempo para ir a hacerme los análisis 😩  \nno tengo el medidor, se me rompió 😕  \nno puedo, no tengo quien me controle 😔  \nno sé, la verdad que estoy medio desorganizado 🤷\u200d♂️  \nno me acuerdo, la última vez fue hace mil años 😅  \nno tengo espacio en mi agenda para eso ahora 💼  \nno tengo las cintas, así que no puedo medir 😬  \nno sé

In [28]:
generados_chat_gpt = [re.sub('\\.$','',mensaje.strip()) for mensaje in respuesta.choices[0].message.content.split("\n")]
print(generados_chat_gpt)

['no, la verdad no sé mis valores 😅', 'no me acuerdo, hace tiempo que no me controlo 🤷\u200d♂️', 'uy, se me pasó che, no lo miré últimamente 😬', 'no tengo tiempo, siempre estoy a mil 💨', 'no conseguí turno en el médico, una garrón 😩', 'no tengo cintas para el medidor, así que no puedo 😕', 'me olvidé de chequearlo, la vida me lleva 😅', 'no tengo el medidor a mano ahora 🤷\u200d♀️', 'hace mucho que no lo miro, no me acuerdo 😅', 'no sé, me faltó ir a la farmacia a comprar cintas 😩', 'no tengo idea, me olvidé de preguntar 😬', 'no pude ir al doc, así que no tengo datos 😔', 'no sé, estoy en otra cosa y no me acuerdo 😅', 'no tengo tiempo para ir a hacerme los análisis 😩', 'no tengo el medidor, se me rompió 😕', 'no puedo, no tengo quien me controle 😔', 'no sé, la verdad que estoy medio desorganizado 🤷\u200d♂️', 'no me acuerdo, la última vez fue hace mil años 😅', 'no tengo espacio en mi agenda para eso ahora 💼', 'no tengo las cintas, así que no puedo medir 😬', 'no sé, tengo que ir a comprar, per

## Parafraseo

In [29]:
import re
# se toman modelos reales no duplicados y de largo menor a 60 caracters
modelos = df["recibido"][df["clase"]=="glicemia"].drop_duplicates()
modelos = [re.sub("\\n"," ",modelo) for modelo in modelos if len(modelo)<60]
modelos

['No lo se a eso .',
 'No sé el valor',
 'No',
 'Hola! Desconozco el valor de mí hemoglobina por ahora',
 'No conozco',
 'No conozco mis valores',
 'Nono no conozco muy bien que significa el valor',
 'No no lo sé.',
 'No tenco como timarmela',
 'No no conosco...',
 'No no se',
 'No sé k es eso',
 'Todavía no me ago el control',
 'No sé todavía no me hago ese control',
 'Hola  Negativo',
 'No lo se',
 'No todavía no puedo aseme los análisis sigo resfriada',
 'No . No sé  cuál es ese valor',
 'No se hace  mucho que no me ago análisis',
 'Buenos dias Disculpe no se el valor qque tengo',
 'Oky necesito el aparato medicion',
 'UD dígame dónde me toman la glucemia',
 'Nose cuánto es',
 'Hla buen día,  quisiera un medidor de glucosa',
 'Buen día; no tengo idea !!!',
 'No conozco, me gustaria saber,como hacer',
 'no',
 'Hla no no conozco el valor',
 'No la verdad que no lo sé',
 'No?',
 'No no se el valor de hemoglobina',
 'Hola no',
 'No veo pero siempre me dicen q.es alta',
 'Buen día.... no

### Mistral AI

In [30]:
prompt = "Parafrasear las siguientes frases para obtener versiones alternativas que mantengan el mismo significado pero con diferentes palabras. Por favor, reformula cada una de las siguientes frases manteniendo en general el modismos y tono pero agregando abreviaturas y errores de tipeo comunes al enviar mensajes de whatsapp así como emoticones acordes en algunas de ellas:\n" + ("\n").join(modelos)
chat_response = client.chat(
    model=model,
    messages=[
        ChatMessage(role="system", content="Do not include explanation in the response, one by line, without numeration"),
        ChatMessage(role="user", temperature = 0.4, content=prompt)
    ]
)

print(chat_response.choices[0].message.content)

No clue atm 🤷‍♀️
No tengo ni idea del valor 😕
Nop 🙅‍♀️
Hola! Aun no se mi nivel de hemoglobina 🩸
No tengo ni 📚
No conozco mis niveles 😟
Nono no estoy seguro de lo que significa el valor 🤔
No no lo se kien sabe 🤷‍♂️
No tengo como checarla 😕
No no conosco... 😟
No no se 🙅‍♀️
No sé k es eso 😕
Todavia no me he checado 🤒
No sé todavía no me chequeo ese nivel 😟
Hola Negativo 🙅‍♂️
No lo se 🤷‍♀️
No todavia no puedo ir por los analisis todavia estoy enferma 🤒
No . No sé cuál es ese nivel 😕
No se hace mucho que no me chequeo 😟
Buenos dias Disculpe no se el nivel qque tengo 😟
Oky necesito el aparato de medicion 📏
UD dígame donde me checan la glucemia 🩸
Nose cuanto es 🤷‍♀️
Hla buen dia, quisiera un medidor de glucosa 📏
Buen dia; no tengo ni idea!!! 😕
No conozco, me gustaria saber,como hacer 🤔
no 🙅‍♀️
Hla no no conozco el nivel 😟
No la verdad que no lo se 🤷‍♂️
No? 🙅‍♀️
No no se el nivel de hemoglobina 😟
Hola no 🙅‍♂️
No veo pero siempre me dicen q.es alta 📈
Buen dia.... no conosco 😟
No Sr/Srta.no con

In [31]:
import re
generados_mistral = [re.sub('\\.$','',mensaje.strip()) for mensaje in chat_response.choices[0].message.content.split("\n")]
nuevos = list(set([texto for texto in generados_mistral if texto not in modelos]))
print(len(modelos))
print(len(nuevos))
nuevos

47
47


['Buenos dias Disculpe no se el nivel qque tengo 😟',
 'Hemoglobina glicosilada 🩸',
 'No lo se. En los proximos analisis lo pediremos. 🤞',
 'No,no se 🤷\u200d♀️',
 'No tengo ni idea del valor 😕',
 'Nop 🙅\u200d♀️',
 'No no se 🙅\u200d♀️',
 'No sé todavía no me chequeo ese nivel 😟',
 'no conozco mi nivel .. 😟',
 'No? 🙅\u200d♀️',
 'No no lo se kien sabe 🤷\u200d♂️',
 'Buen dia; no tengo ni idea!!! 😕',
 'Oky necesito el aparato de medicion 📏',
 'Yo necesito el aparatito que me mido sola todos los dias 📏',
 'Nono no estoy seguro de lo que significa el valor 🤔',
 'Hla no no conozco el nivel 😟',
 'No lo se 🤷\u200d♀️',
 'No Sr/Srta.no conozco el nivel hemogb.glic. 😟',
 'No la verdad que no lo se 🤷\u200d♂️',
 'Hola no tengo el resultado todavia 😟',
 'UD dígame donde me checan la glucemia 🩸',
 'No . No sé cuál es ese nivel 😕',
 'No se hace mucho que no me chequeo 😟',
 'Hola no conosco mi nivel de hemoglobina glicosilada 🩸',
 'No tengo ni 📚',
 'No. 🙅\u200d♀️',
 'No se 🤷\u200d♀️',
 'No conozco mis niv

### ChatGPT

In [32]:
user_prompt = "Parafrasear las siguientes frases para obtener versiones alternativas que mantengan el mismo significado pero con diferentes palabras. Por favor, reformula cada una de las siguientes frases manteniendo en general el modismos y tono pero agregando abreviaturas y errores de tipeo comunes al enviar mensajes de whatsapp así como emoticones acordes en algunas de ellas:\n" + ("\n").join(modelos)
system_prompt = "Do not include explanation in the response, one by line, without numeration"

respuesta = query_chat_gpt(user_prompt, system_prompt, temperature=0.6)

print(respuesta.choices[0].message)

ChatCompletionMessage(content='No lo sé, eso.  \nNo sé el precio.  \nNo.  \nHola! Aún no tengo el dato de mi hemoglobina.  \nNo tengo idea.  \nNo sé mis valores.  \nNono, no entiendo muy bien qué significa ese valor.  \nNo, no lo sé.  \nNo tengo cómo medírmela.  \nNo, no conozco...  \nNo sé.  \nNo sé qué es eso.  \nAún no me hago el control.  \nNo sé, aún no me lo hago.  \nHola, negativo.  \nNo lo sé.  \nNo, todavía no puedo hacerme los análisis, sigo con el resfriado.  \nNo, no sé cuál es ese valor.  \nNo, hace tiempo que no me hago análisis.  \nBuen día, disculpe, no sé el valor que tengo.  \nOky, necesito el aparatito de medición.  \nUsted dígame dónde me miden la glucemia.  \nNo sé cuánto es.  \nHla, buen día, quisiera un medidor de glucosa.  \nBuen día, no tengo idea!!!  \nNo conozco, me gustaría saber cómo hacerlo.  \nNo.  \nHla, no conozco el valor.  \nNo, la verdad no lo sé.  \n¿No?  \nNo sé el valor de hemoglobina.  \nHola, no.  \nNo veo, pero siempre me dicen que es alto.  \n

In [33]:
generados_chat_gpt = [re.sub('\\.$','',mensaje.strip()) for mensaje in respuesta.choices[0].message.content.split(" \n")]
nuevos = list(set([texto for texto in generados_chat_gpt if texto not in modelos]))
print(len(modelos))
print(len(nuevos))
nuevos

47
40


['Buen día, disculpe, no sé el valor que tengo',
 'Usted dígame dónde me miden la glucemia',
 'Hola! Aún no tengo el dato de mi hemoglobina',
 'Hola, aún no tengo el resultado',
 'Hola, no conozco mi valor de hemoglobina glicosilada',
 'Ni idea, ni me controlo el azúcar, no tengo ni pastillas',
 'No tengo cómo medírmela',
 'Hola, no',
 'No, no sé cuál es ese valor',
 'No lo sé',
 'Hla, buen día, quisiera un medidor de glucosa',
 'No lo sé, eso',
 'Oky, necesito el aparatito de medición',
 'No sé el valor de hemoglobina',
 '¿No?',
 'No lo sé. En los próximos análisis lo pediremos',
 'Nono, no entiendo muy bien qué significa ese valor',
 'No, la verdad no lo sé',
 'No, pero lo haré lo antes posible',
 'No sé mis valores',
 'No, no lo sé',
 'No tengo idea',
 'No, no sé',
 'Buen día... no conozco',
 'Aún no me hago el control',
 'No sé cuánto es',
 'Hola, negativo',
 'No, Sr/Srta, no conozco el valor de hemoglobina y glicemia',
 'No, no conozco..',
 'No conozco, me gustaría saber cómo hace

## Backtranslation

In [34]:
import requests

def translate(text, source_lang, target_lang, api_key):
    url = "https://text-translator2.p.rapidapi.com/translate"
    headers = {
        'x-rapidapi-host': "text-translator2.p.rapidapi.com",
        'x-rapidapi-key': api_key,
        'Content-Type': "application/x-www-form-urlencoded"
    }
    payload = {
        'source_language': source_lang,
        'target_language': target_lang,
        'text': text
    }
    response = requests.post(url, headers=headers, data=payload)
    characters_remaining = response.headers.get('x-ratelimit-characters-remaining', 'No limit info')
    return response.json()['data']['translatedText'], characters_remaining

def back_translate(text, intermediate_lang, api_key):
    # Traducir de Español a otro idioma
    translated_text, limit_intermediate = translate(text, source_lang='es', target_lang=intermediate_lang, api_key=api_key)
    
    # Traducir de vuelta al Español
    back_translated_text, limit_back = translate(translated_text, source_lang=intermediate_lang, target_lang='es', api_key=api_key)
    
    return back_translated_text, limit_back

# Ejemplo de uso
api_key = '09d7b40403msh510b48d970ae036p193760jsned3c5571439e'
original_text = "Hola, necesito ayuda para con mi proyecto."
back_translated_text, characters_remaining = back_translate(original_text, intermediate_lang='en', api_key=api_key)
print("Original:", original_text)
print("Back-translated:", back_translated_text)
print("Characters Remaining:", characters_remaining)


Original: Hola, necesito ayuda para con mi proyecto.
Back-translated: Hola, necesito ayuda con mi proyecto.
Characters Remaining: 95234


In [35]:
# se toman modelos reales no duplicados a partir de corregidos y de largo menor a 60 caracters
modelos_corregidos = df["recibido_corregido"][df["clase"]=="glicemia"].drop_duplicates()
modelos_corregidos = [re.sub("\\n"," ",modelo) for modelo in modelos_corregidos if len(modelo)<60]
modelos_corregidos

['no lo se a eso ',
 'no sé el valor',
 'no',
 'hola desconozco el valor de mí hemoglobina por ahora',
 'no conozco',
 'no conozco mis valores',
 'nono no conozco muy bien que significa el valor',
 'no no lo sé ',
 'no tenco como timarmela',
 'no no conozco ',
 'no no se',
 'no sé que es eso',
 'todavía no me hago el control',
 'no sé todavía no me hago ese control',
 'hola negativo',
 'no lo se',
 'no todavía no puedo hacerme los análisis sigo resfriada',
 'no no sé cuál es ese valor',
 'no se hace mucho que no me hago análisis',
 'buenos dias disculpe no se el valor que tengo',
 'ok necesito el aparato medicion',
 'usted dígame dónde me toman la glucemia',
 'nose cuánto es',
 'hola buen día quisiera un medidor de glucosa',
 'buen día no tengo idea ',
 'no conozco me gustaria saber como hacer',
 'hola no no conozco el valor',
 'no la verdad que no lo sé',
 'no ',
 'no no se el valor de hemoglobina',
 'hola no',
 'no veo pero siempre me dicen que es alta',
 'buen día no conozco',
 'no 

In [36]:
resultado_backtranslation = []
for modelo in modelos_corregidos:
    back_translated_text, characters_remaining = back_translate(modelo, intermediate_lang='en', api_key=api_key)
    resultado_backtranslation.append(back_translated_text)
print(resultado_backtranslation)
print(characters_remaining)

['No sé eso', 'No sé el valor', 'No', 'Hola, no sé el valor de mi hemoglobina por ahora', 'No sé', 'No conozco mis valores', 'No sé muy bien lo que significa el valor', 'No, no lo se', 'No tengo timel de mí', 'No sé', 'no no, no lo sé', 'No se que es eso', 'Todavía no controlo', 'Aún no lo sé, hago ese control', 'Hola negativo', 'No sé', 'Todavía no puedo hacer el análisis, soy un resfriado', 'No sé cuál es ese valor', 'No hago mucho que no hago análisis', 'Buenos días, no sé el valor que tengo', 'Ok necesito el dispositivo de medición', 'Dime a dónde llevan glucemia', 'No sé mucho que sea', 'Hola buen día me gustaría un medidor de glucosa', 'Buenos días no tengo idea', 'No sé que me gustaría saber cómo hacer', 'Hola, no sé el valor', 'Realmente no lo sé', 'No', 'No conozco el valor de hemoglobina', 'Hola no', 'No veo pero siempre me dicen que está alto', 'Buen día no lo sé', 'No Sr. Sra. No conozco el Glyc de ValueHemogb', 'No sé', 'Hola, no conozco mi valor de hemoglobina glucosilada

In [37]:
# nuevos
nuevos = list(set([texto for texto in resultado_backtranslation if texto not in modelos_corregidos]))
print(len(modelos_corregidos))
print(len(nuevos))
nuevos

44
39


['No sé mucho que sea',
 'No sé que me gustaría saber cómo hacer',
 'No sé',
 'no no, no lo sé',
 'No lo conozco',
 'Necesito el aparato que me mido solo todos los días',
 'No sé muy bien lo que significa el valor',
 'No Sr. Sra. No conozco el Glyc de ValueHemogb',
 'No sé el valor',
 'ni EDEA ni controlar el azúcar no tengo píldor',
 'Hola, no conozco mi valor de hemoglobina glucosilada',
 'No, no lo se',
 'Todavía no controlo',
 'Hola no',
 'Todavía no puedo hacer el análisis, soy un resfriado',
 'Buenos días, no sé el valor que tengo',
 'Aún no lo sé, hago ese control',
 'no, pero lo haré lo antes posible',
 'Hola, no sé el valor de mi hemoglobina por ahora',
 'No hago mucho que no hago análisis',
 'Hola, todavía no tengo el resultado',
 'No veo pero siempre me dicen que está alto',
 'Buen día no lo sé',
 'Ok necesito el dispositivo de medición',
 'No conozco mi valor',
 'No conozco mis valores',
 'No se que es eso',
 'Hola, no sé el valor',
 'Hola negativo',
 'No',
 'No conozco el 

## Uso de sinónimos

In [38]:
!pip install spacy
!python -m spacy download es_core_news_sm

     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     ---------------------------------------- 0.1/12.9 MB 3.3 MB/s eta 0:00:04
     - -------------------------------------- 0.4/12.9 MB 4.9 MB/s eta 0:00:03
     -- ------------------------------------- 0.7/12.9 MB 5.4 MB/s eta 0:00:03
     --- ------------------------------------ 1.0/12.9 MB 5.6 MB/s eta 0:00:03
     ---- ----------------------------------- 1.3/12.9 MB 5.9 MB/s eta 0:00:02
     ---- ----------------------------------- 1.3/12.9 MB 5.0 MB/s eta 0:00:03
     ----- ---------------------------------- 1.7/12.9 MB 5.3 MB/s eta 0:00:03
     ----- ---------------------------------- 1.9/12.9 MB 5.2 MB/s eta 0:00:03
     ------ --------------------------------- 2.0/12.9 MB 5.0 MB/s eta 0:00:03
     ------ --------------------------------- 2.2/12.9 MB 4.9 MB/s eta 0:00:03
     ------- -------------------------------- 2.3/12.9 MB 4.6 MB/s eta 0:00:03
     ------- -------------------------------- 2.5/12.9 MB 4

In [39]:
!pip install nltk
!python -m nltk.downloader omw-1.4

C:\Users\aciga\anaconda3\envs\nlp\lib\runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\aciga\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [40]:
# Importar la biblioteca NLTK
import nltk

nltk.download('omw-1.4')
nltk.download('wordnet')

from nltk.corpus import wordnet as wn

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\aciga\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aciga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [41]:
!python -m spacy download es_core_news_md

     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
     ---------------------------------------- 0.2/42.3 MB 4.7 MB/s eta 0:00:09
     ---------------------------------------- 0.5/42.3 MB 5.4 MB/s eta 0:00:08
      --------------------------------------- 0.8/42.3 MB 5.9 MB/s eta 0:00:08
     - -------------------------------------- 1.1/42.3 MB 6.0 MB/s eta 0:00:07
     - -------------------------------------- 1.4/42.3 MB 6.1 MB/s eta 0:00:07
     - -------------------------------------- 1.7/42.3 MB 6.0 MB/s eta 0:00:07
     - -------------------------------------- 1.8/42.3 MB 5.5 MB/s eta 0:00:08
     - -------------------------------------- 2.0/42.3 MB 5.6 MB/s eta 0:00:08
     -- ------------------------------------- 2.1/42.3 MB 5.2 MB/s eta 0:00:08
     -- ------------------------------------- 2.3/42.3 MB 4.9 MB/s eta 0:00:09
     -- ------------------------------------- 2.4/42.3 MB 4.9 MB/s eta 0:00:09
     -- ------------------------------------- 2.8/42.3 MB 4

In [42]:
import spacy
import random
from nltk.corpus import wordnet as wn

# Cargar el modelo en español de spaCy
nlp = spacy.load('es_core_news_sm')
###############

def obtener_sinonimos_con_similitud(palabra, pos_spacy):
    wordnet_pos = {'NOUN': 'n', 'VERB': 'v', 'ADJ': 'a', 'ADV': 'r'}.get(pos_spacy, None)
    if wordnet_pos is None:
        return []

    palabra_nlp = nlp(palabra)       
    synsets = wn.synsets(palabra, pos=wordnet_pos, lang='spa')
            
    sinonimos_con_similitud = []

    for synset in synsets:
        for lemma in synset.lemma_names('spa'):
            lemma_nlp = nlp(lemma)
            if lemma_nlp.vector_norm and palabra_nlp.vector_norm:  # Asegurarse de que ambos tengan vectores válidos
                similitud = palabra_nlp.similarity(lemma_nlp)
                sinonimos_con_similitud.append((lemma, round(similitud, 2)))

    return sinonimos_con_similitud


# Función para procesar una frase y sustituir palabras aleatoriamente con sinónimos
def sustituir_con_sinonimos(frase, umbral = 0.5, agregar_por_lemma = False):
    doc = nlp(frase)
    palabras_procesadas = []
    for token in doc:
        sinonimos_con_similitud = obtener_sinonimos_con_similitud(token.text, token.pos_)
        if agregar_por_lemma:
            token_lemma = token.lemma_
            print("lemma",token_lemma)
            palabra_nlp = nlp(token_lemma)[0]                      
            sinonimos_con_similitud = sinonimos_con_similitud + obtener_sinonimos_con_similitud(palabra_nlp.text, palabra_nlp.pos_)
        
        sinonimos = list(set([sinonimo for sinonimo, similitud in sinonimos_con_similitud if similitud >= umbral]))
        print(token,sinonimos_con_similitud, sinonimos)
        if sinonimos:
            palabra_sustituta = random.choice(sinonimos)
            palabras_procesadas.append(palabra_sustituta)
        else:
            palabras_procesadas.append(token.text)
    return ' '.join(palabras_procesadas)

# Aplicar sinónimos
frases_con_sinonimos = [sustituir_con_sinonimos(frase, umbral = 0.5, agregar_por_lemma = True) for frase in modelos_corregidos]
for original, modificada in zip(modelos_corregidos, frases_con_sinonimos):
    print(f"Original: {original}")
    print(f"Modificada: {modificada}\n")


lemma no
no [] []
lemma él
lo [] []
lemma él
se [] []
lemma a
a [] []
lemma ese
eso [] []
lemma no
no [] []
lemma saber
sé [('conocer', 0.41), ('saber', 1.0), ('saber', 1.0), ('conocer', 0.41), ('saber', 1.0), ('saber', 1.0), ('saber', 1.0), ('sentir', 0.54), ('conocer', 0.41), ('saber', 1.0)] ['sentir', 'saber']
lemma el
el [] []


C:\Users\aciga\AppData\Local\Temp\ipykernel_32152\1558089611.py:23: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similitud = palabra_nlp.similarity(lemma_nlp)


lemma valor
valor [('bravura', 0.17), ('coraje', 0.43), ('intrepidez', 0.25), ('osadía', -0.1), ('valor', 1.0), ('coraje', 0.43), ('fuerza', 0.17), ('valor', 1.0), ('ánimo', 0.19), ('atrevimiento', 0.35), ('audacia', 0.28), ('cara', 0.31), ('cara_dura', 0.32), ('osadía', -0.1), ('valor', 1.0), ('coraje', 0.43), ('entereza', 0.34), ('valentía', -0.03), ('valor', 1.0), ('valor', 1.0), ('coraje', 0.43), ('valor', 1.0), ('valor', 1.0), ('agallas', -0.11), ('coraje', 0.43), ('discernimiento', 0.31), ('nervio', 0.56), ('sangre_fría', 0.02), ('valor', 1.0), ('valor', 1.0), ('precio', 0.43), ('valor', 1.0), ('valor', 1.0), ('valor', 1.0), ('coraje', 0.43), ('valor', 1.0), ('valor', 1.0), ('valor', 1.0), ('valor_del_tiempo', 0.33), ('bravura', 0.17), ('coraje', 0.43), ('intrepidez', 0.25), ('osadía', -0.1), ('valor', 1.0), ('coraje', 0.43), ('fuerza', 0.17), ('valor', 1.0), ('ánimo', 0.19), ('atrevimiento', 0.35), ('audacia', 0.28), ('cara', 0.31), ('cara_dura', 0.32), ('osadía', -0.1), ('valor